In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# https://fbref.com/en/squads/b8fd03ef/2023-2024/Manchester-City-Stats
# https://fbref.com/en/squads/18bb7c10/2023-2024/Arsenal-Stats
# https://fbref.com/en/squads/18bb7c10/2023-2024/matchlogs/c9/schedule/Arsenal-Scores-and-Fixtures-Premier-League

In [3]:
BASE_PL_URL='https://fbref.com/en/comps/9/Premier-League-Stats'

In [4]:
result=requests.get(BASE_PL_URL)
soup=BeautifulSoup(result.text,'html.parser')


In [5]:
#user agents rotations
UA=["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0",
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.2420.81",
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ( (KKHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36HTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 14.4; rv:124.0) Gecko/20100101 Firefox/124.0",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0",
"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
"Mozilla/5.0 (X11; Linux i686; rv:124.0) Gecko/20100101 Firefox/124.0"]

In [6]:
# extract the whole table for team names and its features for future urls

In [7]:
# now extract team name and squads id to match 
# https://fbref.com/en/squads/18bb7c10/2023-2024/matchlogs/c9/schedule/Arsenal-Scores-and-Fixtures-Premier-League
# so that we can automate year and reduce scraping complexity to scrape team info based on seasons

def extract_team(year):
    teams={}
    try:
        
        headers={'User-Agent':  random.choice(UA)}    
        result=requests.get(f'https://fbref.com/en/comps/9/{year}-{year+1}/{year}-{year+1}-Premier-League-Stats',headers=headers)
        time.sleep(5)
        soup=BeautifulSoup(result.text,'html.parser')
        table = soup.find('table', id=f'results{year}-{year+1}91_overall')
        teams_urls=[]
        for link in table.findAll('a'):
            if '/squads/' in link['href']:
                teams_urls.append(link)
        for url in teams_urls:
            team_name=url.text.strip()
            team_name_for_url=url['href'].split('/')[-1][:-6]
            team_id=url['href'].split('/')[3]
            # print(team_name,team_id)
            teams[team_name_for_url]=[team_id,team_name]
    except Exception as e:
        print(e)
    return teams


# len(teams)
# teams






In [8]:
# year=2020
# f'results{year}-{year+1}91_overall'
# f'https://fbref.com/en/comps/9/{year}-{year+1}/{year}-{year+1}-Premier-League-Stats'

In [9]:
# now match with 
# https://fbref.com/en/squads/18bb7c10/2023-2024/matchlogs/c9/schedule/Arsenal-Scores-and-Fixtures-Premier-League
# year scraped: 2014-2024   
filename='14-16.csv'   #<<<<<>>>chnge this :scraped only 2 2 years  gap to avoid blockage .IDk you can put many year gap since ua rotation iggg
final_csv=[]
count=0
for year in range(2014,2016):   #<<<<<>>>chnge this
    teams=extract_team(year)
    for name,val in teams.items():
        print(year,year+1,val[1])
        time.sleep(5)
        headers={'User-Agent':  random.choice(UA)}    
        response=requests.get(f'https://fbref.com/en/squads/{val[0]}/{year}-{year+1}/matchlogs/c9/schedule/{name}-Scores-and-Fixtures-Premier-League',headers=headers)
        if response.status_code==200:
            try:
                df=pd.read_html(response.text,match='Scores & Fixtures')[0]
                df['team']=val[1].lower()
                df=df.drop(['Notes'],axis=1)
                df=df.drop(['Match Report'],axis=1)
                df['Date']=pd.to_datetime(df['Date'])
                df['Time']=df['Time'].apply(lambda x : x.split(':')[0])
                final_csv.append(df)
                count=count+1
                print(f'sucess',count)
                
            except Exception as e:
                print(e)
        else:
            print("cant connect")
            
        
            
final=pd.concat(final_csv)
final.to_csv(filename)



2012 2013 Manchester Utd
'float' object has no attribute 'split'
2012 2013 Manchester City
'float' object has no attribute 'split'
2012 2013 Chelsea
'float' object has no attribute 'split'
2012 2013 Arsenal
'float' object has no attribute 'split'
2012 2013 Tottenham
'float' object has no attribute 'split'
2012 2013 Everton
'float' object has no attribute 'split'
2012 2013 Liverpool
'float' object has no attribute 'split'
2012 2013 West Brom


KeyboardInterrupt: 

In [ ]:
pd.read_csv('16-18.csv').shape

In [5]:
38*20*2

1520

# dwadwdad

In [ ]:
r=requests.get('https://fbref.com/en/squads/361ca564/2023-2024/matchlogs/c9/schedule/Tottenham-Hotspur-Scores-and-Fixtures-Premier-League')
r

In [ ]:
team_table_info=pd.read_html(r.text,match='Scores & Fixtures')[0]

In [ ]:
team_table_info.head()

In [ ]:
team_table=team_table_info.drop(['Notes'],axis=1)

In [ ]:
team_table.shape

In [ ]:
team_table.head()

In [ ]:
team_table['Date']=pd.to_datetime(team_table['Date'])
team_table['Time']=team_table['Time'].apply(lambda x : x.split(':')[0])

In [ ]:
team_table.head()

In [7]:
import pandas as pd

df1=pd.read_csv('14-16.csv')
df2=pd.read_csv('16-18.csv')
df3=pd.read_csv('18-20.csv')
df4=pd.read_csv('20-22.csv')
df5=pd.read_csv('22-24.csv')


# Concatenate DataFrames
combined_df = pd.concat([df1, df2,df3,df4,df5], ignore_index=True)



In [8]:
combined_df.to_csv('final2014-2024.csv')

In [9]:

combined_df

,Unnamed: 0,Date,Time,Round,Day,Venue,Result,GF,GA,Opponent,Poss,Attendance,Captain,Formation,Referee,team,xG,xGA
0,0,2014-08-18,20,Matchweek 1,Mon,Away,W,3,1,Burnley,61,20699.0,NaN,4-2-3-1,Michael Oliver,chelsea,NaN,NaN
1,1,2014-08-23,15,Matchweek 2,Sat,Home,W,2,0,Leicester City,63,41604.0,NaN,4-2-3-1,Lee Mason,chelsea,NaN,NaN
2,2,2014-08-30,17,Matchweek 3,Sat,Away,W,6,3,Everton,39,39402.0,NaN,4-2-3-1,Jonathan Moss,chelsea,NaN,NaN
3,3,2014-09-13,15,Matchweek 4,Sat,Home,W,4,2,Swansea City,55,41400.0,NaN,4-2-3-1,Kevin Friend,chelsea,NaN,NaN
4,4,2014-09-21,16,Matchweek 5,Sun,Away,D,1,1,Manchester City,42,45602.0,NaN,4-2-3-1,Mike Dean,chelsea,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,33,2024-04-24,20,Matchweek 29,Wed,Away,L,2,4,Manchester Utd,29,73549.0,Oliver Arblaster,3-5-2,Michael Salisbury,sheffield utd,0.8,2.9
7596,34,2024-04-27,15,Matchweek 35,Sat,Away,L,1,5,Newcastle Utd,40,52196.0,Oliver Arblaster,3-5-2,Tony Harrington,sheffield utd,1.5,3.1
7597,35,2024-05-04,15,Matchweek 36,Sat,Home,L,1,3,Nott'ham Forest,54,28516.0,Jack Robinson,3-5-2,Chris Kavanagh,sheffield utd,2.2,1.5
7598,36,2024-05-11,15,Matchweek 37,Sat,Away,L,0,1,Everton,42,39221.0,Jack Robinson,3-5-2,Stuart Attwell,sheffield utd,0.6,2.3


In [ ]:
1520*